# Neem Evaluation

This example will provide you with an introduction to the evaluation of NEEMs. The evaluation of this package aims at the behaviour captured by the NEEM. 

There are two major parts of a NEEM which capture the events and behaviour in a NEEM. These parts are the NEEM experience and NEEM behaviour, the experience are the TF data of every object at every time while the behaviour are the semantic events that happend over the span of the NEEM. Both of them are evaluated seperatly. 



## Install Neem_evaluator
You need to have the Neem_evaluator python package installed for this example to work. 

To install the Neem_evaluator package navigate to the directory of the Neem_evaluator, open a terminal and enter the following command:
~~~
pip install .
~~~


## Loading a NEEM

To evaluate a NEEM we need to load it first. There are two things that need to be loaded the first is the NEEM to knowrob, this needs to be done to query the NEEM and to access the data. The second one is the Neem class of the Neem evaluator, this is the basis for every metric that will be applied. The Neem class is a more simplistic representation of a NEEM as a Python data structure. 

To load the NEEM to Knowrob you need to start Knowrob first. You can start Knowrob by typing the following command into a terminal. 
~~~
roslaunch knowrob knowrob.launch
~~~

To load a saved NEEM to knowrob we can use the ```remember_neem``` method which takes the path to the saved NEEM as an argument

In [10]:
from neem_evaluator.knowrob import remember_neem

remember_neem("/home/jdech/workspace/NEEM_Evaluator/PouringNEEMData/ApartmentLabNEEM")

Lastly, we need to create an instance of the Neem class.  The Neem class is an internal representation of a NEEM loaded in Knowrob, this is done for easier use since it allows to treat a NEEM as a Python data structure.

In [11]:
from neem_evaluator.neem import Neem

n = Neem()

[INFO] [1702981633.522167]: NEEM Object: NaturalPerson_CTYBCZBF has no tf_link_name therefore no tf pointer could be loaded
[INFO] [1702981633.587903]: NEEM Object: NaturalPerson_CTYBCZBF has no tf_link_name therefore no tf pointer could be loaded
[INFO] [1702981633.651309]: NEEM Object: NaturalPerson_CTYBCZBF has no tf_link_name therefore no tf pointer could be loaded
[INFO] [1702981633.679528]: NEEM Object: NaturalPerson_CTYBCZBF has no tf_link_name therefore no tf pointer could be loaded
[INFO] [1702981633.706342]: NEEM Object: NaturalPerson_CTYBCZBF has no tf_link_name therefore no tf pointer could be loaded
[INFO] [1702981633.732178]: NEEM Object: NaturalPerson_CTYBCZBF has no tf_link_name therefore no tf pointer could be loaded
[INFO] [1702981633.756858]: NEEM Object: NaturalPerson_CTYBCZBF has no tf_link_name therefore no tf pointer could be loaded
[INFO] [1702981633.805200]: NEEM Object: TopRightDirection_1 has no tf_link_name therefore no tf pointer could be loaded
[INFO] [170

## Neem Experience

We start by taking a look at the NEEM experience, since these are basically the trajectories of objects over the span of the NEEM we can evaluate how "smooth" a trajectory is. This is done by checking how often the direction of the trajectory is changing. 

In [24]:
from neem_evaluator.metrics import vel_metric

hand_object = n.get_objects_by_name("right_hand_1")
metric, sequences = vel_metric(hand_object.get_tfs(), return_seq=True)

print(f"The result of the metric is: {metric}")
print(f"The sequences where the metric was triggered are: {sequences}")

The result of the metric is: 73
The sequences where the metric was triggered are: [  0 140 141 142 143 152 153 154 155 156 157 158 159 160 161 162 163 164
 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182
 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259
 296 297 298 299 300 301 302 303 312 313 314 315 316 317 318 319 336 337
 338 339 340 341 342 343 344 345 346 347 348 349 350 351 378 379 380 381
 382 383 384 385 386 387 390 391 392 393 394 395 396 397 398 399 400 401
 402 403 404 405 406 407]


What you get from the velocity metric method are two things, the first is the number of times the metric was triggered meaning the number of times where the change in the direction was bigger than a specific threshold. The second one are the sequence numbers of TF frames where the metric was triggered. We can use these sequence numbers to visualize where the metric was triggered. Upon closer inspection you will find that the sequences are usually in clusters where a bunch of sequences are consconsecutively.


## Neem Behaviour
Since the behaviour are the semantic events that happend during the NEEM and evaluating them needs a lot of contextual information. Meaning how good or bad a certain metric is can highly depend on the behaviour that is recorded in the NEEM, since this makes it especially hard to evaluate the semantic events we opted for some sort of "expert-system" where a bunch of different metrics will be applied to the NEEM behaviour and presented as-is. 

It is then the task of the user to check which metrics are applicable to the current NEEM and use them for evaluation. 

As already stated, behaviour evaluation is extremly context dependend. As an attempt to incorporate context into the metrics all metrics compare the NEEM which should be evaluated to an "optimal" NEEM which was hand-picked by the evaluator as an example on how the task should be performed. Every other NEEM is then evaluated by its deviation from this "optimal" NEEM.

Since Knowrob can only load one NEEM at the time but we need two NEEMs for the behaviour evaluation we developed a serialization mechanism for NEEMs which allows to store and load NEEMs as json files. To serialize a NEEM you need to load it into Knowrob and create an instance of the Neem class, then just call ``save(filepath)`` on the instance and the NEEM will be serialized and stored. 


In [26]:
n.save("stored_NEEM.json")

To load a serilized NEEM from a json file just use the path to the json file as an argument when creating the instance of the Neem class. 

In [ ]:
loaded_neem = Neem("stored_NEEM.json")

Now that we have two NEEMs, one from Knowrob and one loaded from a json file, we can use the respective metric to evaluate the NEEM behaviour between these two. 

In [25]:
from neem_evaluator.events import event_metric

event_metric(n, n)

AttributeError: 'Neem' object has no attribute 'action_tf_offset'